In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
file_url = "C:\\Users\\i.bermejo\\Documents\\Data\\MedicationReconciliation\\PreprocessedData.csv"
dataframe = pd.read_csv(file_url)

In [46]:
dataframe.shape

(1526, 180)

In [47]:
dataframe.head()

,id,sex,age,n_diseases,diseases,eGFR,n_visits_out_all_12m_MUMC,in_all_12m_MUMC,n_ER_12m_MUMC,n_visits_out_current_specialty_12m_MUMC.cat,...,prevented_consequences.F_1,prevented_consequences.G_1,prevented_consequences.H_1,prevented_consequences.I_1,information_gained.yes_direct_1,information_gained.yes_indirect_1,information_gained.no_1,y.interaction,y.revision_wo_interaction,y
0,w7DXu1IzfC,female,56,NaN,NaN,60-89 ml/min,> 5,no,0,3+,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0,0,0
1,kkcuBd72WA,female,68,3.0,"1,3,4",15-29 ml/min,1,yes,2,1,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0,0,0
2,O8Ss3eOnMD,female,36,NaN,NaN,unknown or measured more than 12 months ago,1,no,0,1,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0,0,0
3,ntN2M82pwD,female,71,1.0,2,30-59 ml/min,> 5,no,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,Mmq8YLjTdy,male,49,NaN,NaN,> 90 ml/min,0,no,0,3+,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0,0,0


In [48]:
dataframe.columns

Index(['id', 'sex', 'age', 'n_diseases', 'diseases', 'eGFR',
       'n_visits_out_all_12m_MUMC', 'in_all_12m_MUMC', 'n_ER_12m_MUMC',
       'n_visits_out_current_specialty_12m_MUMC.cat',
       ...
       'prevented_consequences.F_1', 'prevented_consequences.G_1',
       'prevented_consequences.H_1', 'prevented_consequences.I_1',
       'information_gained.yes_direct_1', 'information_gained.yes_indirect_1',
       'information_gained.no_1', 'y.interaction', 'y.revision_wo_interaction',
       'y'],
      dtype='object', length=180)

In [49]:
dataframe['in_all_12m_MUMC'].unique()

array(['no', 'yes'], dtype=object)

In [50]:
dataframe.dtypes

id                                    object
sex                                   object
age                                    int64
n_diseases                           float64
diseases                              object
                                      ...   
information_gained.yes_indirect_1    float64
information_gained.no_1              float64
y.interaction                          int64
y.revision_wo_interaction              int64
y                                      int64
Length: 180, dtype: object

In [85]:
dataframe['n_diseases'] = dataframe['n_diseases'].fillna(0).astype(int)
dataframe['n_visits_out_current_specialty_12m_MUMC_1'] =  np.where(dataframe['n_visits_out_current_specialty_12m_MUMC.cat']== '1', 1, 0)
dataframe['n_visits_out_current_specialty_12m_MUMC_2'] =  np.where(dataframe['n_visits_out_current_specialty_12m_MUMC.cat']== '2', 1, 0)
dataframe['n_visits_out_current_specialty_12m_MUMC_3plus'] =  np.where(dataframe['n_visits_out_current_specialty_12m_MUMC.cat']== '3+', 1, 0)



In [89]:
df = dataframe.drop(['id', 
                     'sex', 
                     'diseases', 
                     'eGFR', 
                     'in_all_12m_MUMC',
                     'n_visits_out_all_12m_MUMC',
                     'n_visits_out_current_specialty_12m_MUMC.cat',
                     'high_risk_medications',
                     'housing',
                     'patient_reported_medication_use', 
                     'pill_box.week',
                     'education', 
                     'allergy_medication',
                     'visit_12m_external_hospital',
                     'in_all_12m_external_hospital',
                     'allergy_medication_symptoms', 
                     'medication_prescribed_during_consult.name', 
                     'out_specialty_current_MUMC',
                     'age.cat', 
                     'y.interaction', 
                     'y.revision_wo_interaction', 
                     'person_responsible_for_medications', 
                     'medication_wo_prescription',
                     'medication_stopped_during_consult.name',
                     'prevented_consequences',
                    'information_gained'], axis=1)

In [96]:
df.shape

df['medication_literacy'].value_counts()

6

In [ ]:
df['medication_literacy'].isna().sum()

Preprocessing

In [91]:
for column in df.columns:
    print(column, ":", df[column].unique())

age : [ 56  68  36  71  49  50  62  70  44  81  72  43  41  76  69  67  60  92
  73  84  90  61  80  57  75  58  78  37  79  59  82  55  66  64  74  51
  83  52  38  46  26  53  54  63  45  20  23  16  10  19  39  30  32   1
  40  89  18  47  91  77  42  48  94  65  85  35  25  22 100   6   4  27
  33   9  11  87  88  96  86  34  31  28  21  24  14  15   0  17  95  93
  29   8  13   7]
n_diseases : [0 3 1 2 4 5 6]
n_ER_12m_MUMC : [ 0  2  1  3  4  5  6 12 20]
n_medications : [ 1 18  5 13  0  2  3 19 14 11  9  8 10  6  7  4 15 17 16 12 20]
n_ER_12m_external_hospital : [ 0  3  1  2  4 20 15]
medication_literacy : ['adequate' 'suboptimal' 'insufficient' nan]
medication_prescribed_during_consult.yes_no : ['1' '2' 'yes']
medication_prescribed_during_consult.new : ['yes' 'no']
medication_stopped_during_consult.yes_no : ['no' 'yes' nan]
non_medication_intervention_during_consult : ['yes' 'no' nan]
warnings : ['no' nan 'yes']
medications_interaction : [nan 'no' 'yes']
medications_revision.physi

In [11]:
# Categorical features encoded as integers
sex = keras.Input(shape=(1,), name="sex.male_1", dtype="int64")

# Numerical features
age = keras.Input(shape=(1,), name="age")
n_diseases  = keras.Input(shape=(1,), name="n_diseases")
n_ER_12m_MUMC = keras.Input(shape=(1,), name="n_ER_12m_MUMC")
n_medications = keras.Input(shape=(1,), name="n_medications")
n_ER_12m_external_hospital = keras.Input(shape=(1,), name="n_ER_12m_external_hospital")

all_inputs = [
    sex,
    cp,
    fbs,
    restecg,
    exang,
    ca,
    thal,
    age,
    trestbps,
    chol,
    thalach,
    oldpeak,
    slope,
]

In [60]:

def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [ ]:
all_features = layers.concatenate(
    [
        sex_encoded,
        cp_encoded,
        fbs_encoded,
        restecg_encoded,
        exang_encoded,
        slope_encoded,
        ca_encoded,
        thal_encoded,
        age_encoded,
        trestbps_encoded,
        chol_encoded,
        thalach_encoded,
        oldpeak_encoded,
    ]

In [ ]:
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_ds, epochs=50, validation_data=val_ds)